In [8]:
import sys
import psycopg2
import datetime
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine 
from sqlalchemy import text
import geoalchemy2
from shapely import wkt
import matplotlib
import gc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, roc_auc_score, f1_score, average_precision_score, fbeta_score
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, roc_curve, RocCurveDisplay
from sklearn.neighbors import KNeighborsClassifier
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall, roc_curve
import shap
import sklearn
import joblib
#activate high interaction shell so print() is not necessary to show output. https://stackoverflow.com/questions/31764006/ipython-notebook-display-every-line-output-without-print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#1. Get data
vmsOp2 = pd.read_csv('data/vmsOp2.csv')
vmsOp2 = vmsOp2.loc[vmsOp2['Date']>'2021-01-01'].copy()

#2. Balance dataset between fishing operations (A, C, N)
print('Unbalanced dataset structure:')
vmsOp2.groupby('Operation').count()

#sample same number of points as smallest category(C)
n_bal = len(vmsOp2.loc[vmsOp2['Operation'] == 'C'])
C_bal = vmsOp2.loc[vmsOp2['Operation']=='C'] #get all points for C
A_bal = vmsOp2.loc[vmsOp2['Operation']=='A'].sample(n_bal)
N_bal = vmsOp2.loc[vmsOp2['Operation']=='N'].sample(n_bal)

print('Balanced dataset structure:')
vmsOp2_bal = pd.concat([C_bal, A_bal, N_bal])
vmsOp2_bal.groupby('Operation').count()


Unbalanced dataset structure:


,Code,Date,Latitude,Longitude,geom,Speed,Course,CourseCorrected,TrackCode,Effort,...,IniDate,EndDate,DayTime,DayTime2,geometry,speedDiff-1,speedDiff+1,cogDiff,bufferGeom,bufferCount
Operation,,,,,,,,,,,,,,,,,,,,,
A,565,565,565,565,565,565,565,565,565,565,...,565,565,565,565,565,565,565,565,565,565
C,440,440,440,440,440,440,440,440,440,440,...,440,440,440,440,440,440,440,440,440,440
N,992,992,992,992,992,992,992,992,992,992,...,992,992,992,992,992,992,992,992,992,992


Balanced dataset structure:


,Code,Date,Latitude,Longitude,geom,Speed,Course,CourseCorrected,TrackCode,Effort,...,IniDate,EndDate,DayTime,DayTime2,geometry,speedDiff-1,speedDiff+1,cogDiff,bufferGeom,bufferCount
Operation,,,,,,,,,,,,,,,,,,,,,
A,440,440,440,440,440,440,440,440,440,440,...,440,440,440,440,440,440,440,440,440,440
C,440,440,440,440,440,440,440,440,440,440,...,440,440,440,440,440,440,440,440,440,440
N,440,440,440,440,440,440,440,440,440,440,...,440,440,440,440,440,440,440,440,440,440


In [19]:
#3. Split dataset into response (y) and predictive variables (x)
finalDF = vmsOp2_bal
finalDF.to_csv('data/balanced_dataset.csv')
x = finalDF[['Speed', 'CourseCorrected', 'cogDiff', 'speedDiff+1', 'speedDiff-1', 'DayTime2', 'bufferCount']]
y = finalDF.Operation

#encode labels 
le = LabelEncoder()
y = pd.Series(le.fit_transform(y))
print('labels encoded as:')
dict(zip(le.classes_, le.transform(le.classes_)))

#4. Split x, y datasets into training and validation 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
print('vmsOp2 balanced total len: ' + str(len(finalDF)))
print('vmsOp2 balanced training len: ' + str(len(y_train)))
print('vmsOp2 balanced test len: ' + str(len(y_test)))
y_test.value_counts()

#5. Check preliminar model accuracy (without hyperparameter tunning)
model = RandomForestClassifier(n_estimators=300, max_features=1)
model.fit(x_train, y_train) 
print('The model accuracy in the training dataset is: ' + str(model.score(x_train, y_train)))
print('The model accuracy in the testing dataset is: ' + str(model.score(x_test, y_test)))


labels encoded as:


{'A': 0, 'C': 1, 'N': 2}

vmsOp2 balanced total len: 1320
vmsOp2 balanced training len: 990
vmsOp2 balanced test len: 330


0    116
1    114
2    100
dtype: int64

RandomForestClassifier(max_features=1, n_estimators=300)

The model accuracy in the training dataset is: 1.0
The model accuracy in the testing dataset is: 0.7212121212121212
